<a href="https://colab.research.google.com/github/iam-yeong/my-first-web-site/blob/main/%EC%A0%9C%ED%92%88_%EC%84%A0%EC%A0%95_%EB%B0%8F_%EC%A0%84%EB%9E%B5_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 카테고리 별 판매량
value_counts.plot(kind='bar')
plt.title('Frequency of Unique Values in Category')
plt.xlabel('Unique Values')
plt.ylabel('Frequency')
plt.savefig('category', dpi=300)
plt.show()

In [ ]:
# 카테고리 별 가격 분포
plt.figure(figsize=(12, 10))
sns.boxplot(x='price', y='category', data=pakistan_data, showfliers=False)
plt.title('Price Distribution by Category')
plt.xlabel('Price')
plt.ylabel('Category')
plt.xlabel('Price')
# 가격 범위가 넓기 때문에 로그 스케일을 적용합니다.
plt.xscale('log')

plt.show()

In [ ]:
# 카테고리 별 총 매출

category_amount_sum = pakistan_group.groupby('category')['grand_total'].sum()
status_amount = pakistan_group.groupby(['category', 'status'])['grand_total'].sum().unstack(fill_value=0)
status_amount['Success Amount'] = status_amount['complete'] - (status_amount['refund'])

# 3. 그래프 그리기
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# 첫 번째 그래프: 카테고리별 총 'amount' 합계
category_amount_sum.plot(kind='bar', ax=axes[0], color='blue')
axes[0].set_title('Total Amount by Category')
axes[0].set_ylabel('Total Amount')
axes[0].set_xlabel('Category')

plt.tight_layout()
plt.show()

In [ ]:
# 카테고리 별 주문 상태 분포

sns.set_style("whitegrid")


category_status_counts = pd.pivot_table(pakistan_group, values='increment_id', index='category', columns='status', aggfunc='count', fill_value=0)


plt.figure(figsize=(14, 8))
sns.heatmap(category_status_counts, annot=True, fmt="d", cmap="YlGnBu")
plt.title('Category vs. Order Status')
plt.xlabel('Order Status')
plt.ylabel('Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 카테고리 성공 주문 비율
status_counts = pakistan_group.groupby(['category', 'status'])['item_id'].count().unstack(fill_value=0)

status_counts['Total Orders'] = status_counts.sum(axis=1)


status_counts['Success Rate'] = status_counts['complete'] / status_counts['Total Orders']


status_counts_sorted = status_counts.sort_values('Success Rate', ascending=False)


plt.figure(figsize=(12, 8))
status_counts_sorted['Success Rate'].plot(kind='bar', color='skyblue')
plt.title('Category - Order Success Rate')
plt.xlabel('Category')
plt.ylabel('Success Rate')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# 가격대 별 취소와 환불 비중
filtered_data = pakistan_group[parkistan_group['status'].isin(['complete', 'refund', 'canceled'])]
price_bins = [0, 1000, 5000, 10000, 20000, 50000, 100000, float('inf')]
price_labels = ['0-1000', '1001-5000', '5001-10000', '10001-20000', '20001-50000', '50001-100000', '100000+']
filtered_data['price_range'] = pd.cut(filtered_data['price'], bins=price_bins, labels=price_labels, right=False)


price_range_status_counts = filtered_data.groupby(['price_range', 'status'])['item_id'].count().unstack(fill_value=0)

refund_counts = price_range_status_counts.get('refund', pd.Series(0, index=price_range_status_counts.index))
canceled_counts = price_range_status_counts.get('canceled', pd.Series(0, index=price_range_status_counts.index))
complete_counts = price_range_status_counts.get('complete', pd.Series(0, index=price_range_status_counts.index))


total = refund_counts + canceled_counts + complete_counts
price_range_status_counts['Refund Rate'] = refund_counts / total
price_range_status_counts['Canceled Rate'] = canceled_counts / total


plt.figure(figsize=(12, 8))
price_range_status_counts[['Refund Rate', 'Canceled Rate']].plot(kind='bar', color=['purple', 'orange'])
plt.title('Refund and Canceled Rates by Price Range')
plt.xlabel('Price Range')
plt.ylabel('Rate')
plt.xticks(rotation=45)
plt.legend(['Refund Rate', 'Canceled Rate'])
plt.tight_layout()
plt.show()

In [ ]:
# 상품 카테고리에 따른 결제 수단 비율
category_payment_counts = parkistan_group.groupby(['category', 'payment_method']).size().unstack(fill_value=0)
category_payment_proportion = category_payment_counts.divide(category_payment_counts.sum(axis=1), axis=0)


ax = category_payment_proportion.plot(kind='bar', stacked=True, figsize=(10, 7), colormap='tab20')


ax.set_title('Payment Method by Category')
ax.set_xlabel('Category')
ax.set_ylabel('Rate')


plt.xticks(rotation=90)


plt.legend(title='Payment Method', bbox_to_anchor=(1.05, 1), loc='upper left')


plt.tight_layout()
plt.show()

In [ ]:
# 카테고리별 월별 할인 추이
top_categories = ['Mobiles & Tablets', "Men's Fashion", "Women's Fashion", 'Appliances', 'Superstore', 'Beauty & Grooming', 'Soghaat']
bottom_categories = ['Home & Living', 'Kids & Baby', '\\N', 'Others', 'Entertainment', 'Computing', 'Health & Sports', 'Books']


top_data = pakistan_group[pakistan_group['category'].isin(top_categories)]
bottom_data = pakistan_group[pakistan_group['category'].isin(bottom_categories)]


top_monthly_price = top_data.groupby(['Month', 'category'])['discount_amount'].sum().unstack()
bottom_monthly_price = bottom_data.groupby(['Month', 'category'])['discount_amount'].sum().unstack()


fig, ax = plt.subplots(2, 1, figsize=(15, 10), sharex=True)


top_monthly_price.plot(ax=ax[0])
ax[0].set_title('Top Categories: Discount by Month')
ax[0].set_ylabel('Discount')
ax[0].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=90)

bottom_monthly_price.plot(ax=ax[1])
ax[1].set_title('Bottom Categories: Discount by Month')
ax[1].set_ylabel('Discount')
ax[1].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=90)



plt.show()


In [ ]:
# 카테고리별 월별 판매량 추이

pakistan_group['Month'] = pakistan_group['created_at'].dt.strftime('%Y-%m')

monthly_order_data = pakistan_group.groupby(['Month', 'category'])['increment_id'].nunique().reset_index()
monthly_order_data.rename(columns={'increment_id': 'Order Count'}, inplace=True)


top_categories = ['Mobiles & Tablets', "Men's Fashion", "Women's Fashion", 'Appliances', 'Superstore', 'Beauty & Grooming', 'Soghaat']
bottom_categories = ['Home & Living', 'Kids & Baby', '\\N', 'Others', 'Entertainment', 'Computing', 'Health & Sports', 'Books']


top_data = monthly_order_data[monthly_order_data['category'].isin(top_categories)]


bottom_data = monthly_order_data[monthly_order_data['category'].isin(bottom_categories)]


sns.set_theme(style="whitegrid")


plt.figure(figsize=(15, 5))
sns.lineplot(data=top_data, x='Month', y='Order Count', hue='category')
plt.xticks(rotation=90)
plt.title('Top 7 Category Volume')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


plt.figure(figsize=(15, 5))
sns.lineplot(data=bottom_data, x='Month', y='Order Count', hue='category')
plt.xticks(rotation=90)
plt.title('Bottom Category Volume')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.show()ow()